# DESPLIEGUE DE RNC PARA PREDICCIÓN DEL TIPO DE GRANO DE CAFÉ

### Integrantes

- Daniel Ortiz Aristizábal
- Felipe Torres Montoya
- Samuel Betancur Muñoz

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from PIL import Image
import os
import torch
import torch.nn as nn
import torch.nn.functional as F
from torchvision import transforms
import pickle
from sklearn.preprocessing import LabelEncoder

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F

class CNN(nn.Module):
    def __init__(self, in_channels, num_classes):
        super(CNN, self).__init__()
        self.conv1 = nn.Conv2d(in_channels, 16, kernel_size=3, padding=1) # Conv 1 de 16 kernels de tamaño 3x3
        self.pool1 = nn.MaxPool2d(kernel_size=2, stride=2) # Pooling (128x128x16)

        self.conv2 = nn.Conv2d(16, 32, kernel_size=3, padding=1) # Conv 2 de 32 kernels de tamaño 3x3
        self.pool2 = nn.MaxPool2d(kernel_size=2, stride=2) # Pooling (64x64x32)

        self.flatten = nn.Flatten() # Flatten para convertir a formato unidimensional (64x64x32 = 131.072 elementos por imagen)
        self.dropout = nn.Dropout(p=0.2) # Dropout del 20% para evitar overfitting

        self.fc1 = nn.Linear(64 * 64 * 32, 100) # Capa oculta, recibe los 131.072 elementos y genera una salida de 100 neuronas
        self.fc2 = nn.Linear(100, num_classes)  # Capa de salida (no se usa softmax porque CrossEntropyLoss lo aplica internamente)

    def forward(self, x):
        x = F.relu(self.conv1(x))
        x = self.pool1(x)
        x = F.relu(self.conv2(x))
        x = self.pool2(x)
        x = self.flatten(x)
        x = F.relu(self.fc1(x))
        x = self.dropout(x)
        x = self.fc2(x)
        return x

# Establecer el dispositivo e inicializar el modelo
device = "cuda" if torch.cuda.is_available() else "cpu"
model = CNN(in_channels=3, num_classes=5).to(device)
print(model)

CNN(
  (conv1): Conv2d(3, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (pool1): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (conv2): Conv2d(16, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (pool2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (dropout): Dropout(p=0.2, inplace=False)
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (fc1): Linear(in_features=131072, out_features=100, bias=True)
  (fc2): Linear(in_features=100, out_features=5, bias=True)
)


In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Usando dispositivo: {device}")

Usando dispositivo: cuda


In [ ]:
model_deep = CNN(in_channels=3, num_classes=5).to(device)

In [ ]:
model_deep.load_state_dict(torch.load('model_deep.pth', map_location=device))

<All keys matched successfully>

In [ ]:
model_deep.eval()
print("Modelo cargado exitosamente.")

Modelo cargado exitosamente.


In [ ]:
print(model_deep)

CNN(
  (conv1): Conv2d(3, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (pool1): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (conv2): Conv2d(16, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (pool2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (dropout): Dropout(p=0.2, inplace=False)
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (fc1): Linear(in_features=131072, out_features=100, bias=True)
  (fc2): Linear(in_features=100, out_features=5, bias=True)
)


In [ ]:
# Cargar el LabelEncoder
filename = 'labelencoder.pkl'
labelencoder = pickle.load(open(filename, 'rb'))
print("LabelEncoder cargado exitosamente.")

LabelEncoder cargado exitosamente.


In [ ]:
# Definir transformaciones para preprocesamiento
preprocess_transforms = transforms.Compose([
    transforms.Resize((256, 256)),  # Tamaño que espera la red
    transforms.ToTensor(), # Convierte imagen de [0,255] a [0,1]
])

# Función para preprocesar una imagen
def preprocess_image(imagen):
    imagen = preprocess_transforms(imagen)
    imagen = imagen.unsqueeze(0)
    return imagen

In [ ]:
# Lectura de la carpeta con las imágenes
path = "futuro"
files = os.listdir(path)

futuro = []
image_names = []

for f in files:
    if f.endswith(".jpg"):
        imagen = Image.open(os.path.join(path, f)).convert('RGB')
        print(f)
        imagen_tensor = preprocess_image(imagen)
        futuro.append(imagen_tensor)
        image_names.append(f)

pinton1.jpg
pinton4.jpg
seco2.jpg
seco1.jpg
maduro3.jpg
maduro6.jpg
maduro4.jpg
sobremaduro1.jpg
sobremaduro4.jpg
maduro1.jpg
seco3.jpg
maduro5.jpg
maduro2.jpg
pinton6.jpg
verde2.jpg
verde1.jpg
sobremaduro2.jpg
sobremaduro3.jpg
seco4.jpg
seco5.jpg


In [ ]:
# Convertir la lista de imágenes en un tensor
img_futuro = torch.cat(futuro, dim=0).to(device)

In [ ]:
# Realizar la predicción
with torch.no_grad():
    predicciones = model_deep(img_futuro)
    _, etiquetas_predichas = torch.max(predicciones, dim=1)
    etiquetas_predichas = etiquetas_predichas.cpu().numpy()

In [ ]:
# Decodificar las etiquetas
etiquetas_decodificadas = labelencoder.inverse_transform(etiquetas_predichas)

In [ ]:
# Imprimir resultados
print("\nPredicciones:")
for nombre, etiqueta in zip(image_names, etiquetas_decodificadas):
    print(f"Imagen: {nombre} -> Clase predicha: {etiqueta}")


Predicciones:
Imagen: pinton1.jpg -> Clase predicha: pintones
Imagen: pinton4.jpg -> Clase predicha: maduros
Imagen: seco2.jpg -> Clase predicha: secos
Imagen: seco1.jpg -> Clase predicha: secos
Imagen: maduro3.jpg -> Clase predicha: maduros
Imagen: maduro6.jpg -> Clase predicha: maduros
Imagen: maduro4.jpg -> Clase predicha: maduros
Imagen: sobremaduro1.jpg -> Clase predicha: sobremaduros
Imagen: sobremaduro4.jpg -> Clase predicha: maduros
Imagen: maduro1.jpg -> Clase predicha: maduros
Imagen: seco3.jpg -> Clase predicha: secos
Imagen: maduro5.jpg -> Clase predicha: maduros
Imagen: maduro2.jpg -> Clase predicha: maduros
Imagen: pinton6.jpg -> Clase predicha: maduros
Imagen: verde2.jpg -> Clase predicha: verdes
Imagen: verde1.jpg -> Clase predicha: verdes
Imagen: sobremaduro2.jpg -> Clase predicha: sobremaduros
Imagen: sobremaduro3.jpg -> Clase predicha: sobremaduros
Imagen: seco4.jpg -> Clase predicha: secos
Imagen: seco5.jpg -> Clase predicha: secos


# Despliegue Optimizado


In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F

class CNN_Optimized(nn.Module):
    def __init__(self, in_channels, num_classes):
        super(CNN_Optimized, self).__init__()
        self.conv1 = nn.Conv2d(in_channels, 16, kernel_size=3, padding=1) # Conv 1 de 16 kernels de tamaño 3x3
        self.bn1 = nn.BatchNorm2d(16)
        self.pool1 = nn.MaxPool2d(kernel_size=2, stride=2) # Pooling (128x128x16)

        self.conv2 = nn.Conv2d(16, 32, kernel_size=3, padding=1) # Conv 2 de 32 kernels de tamaño 3x3
        self.bn2 = nn.BatchNorm2d(32)
        self.pool2 = nn.MaxPool2d(kernel_size=2, stride=2) # Pooling (64x64x32)

        self.conv3 = nn.Conv2d(32, 48, kernel_size=3, padding=1) # Conv 3 de 48 kernels de tamaño 3x3
        self.bn3 = nn.BatchNorm2d(48)
        self.pool3 = nn.MaxPool2d(kernel_size=2, stride=2) # Pooling (32x32x48)

        self.flatten = nn.Flatten() # Flatten para convertir a formato unidimensional (32x32x48 = 49.152 elementos por imagen)
        self.dropout = nn.Dropout(p=0.2) # Dropout del 20% para evitar overfitting

        self.fc1 = nn.Linear(32 * 32 * 48, 70) # Capa oculta 1, recibe los 49.152 elementos y genera una salida de 70 neuronas
        self.fc2 = nn.Linear(70, 35) # Capa Oculta 2 con 35 neuronas
        self.fc3 = nn.Linear(35, num_classes)  # Capa de salida (no se usa softmax porque CrossEntropyLoss lo aplica internamente)

    def forward(self, x):
        x = self.pool1(F.relu(self.bn1(self.conv1(x))))
        x = self.pool2(F.relu(self.bn2(self.conv2(x))))
        x = self.pool3(F.relu(self.bn3(self.conv3(x))))
        x = self.flatten(x)
        x = self.dropout(F.relu(self.fc1(x)))
        x = self.dropout(F.relu(self.fc2(x)))
        x = self.fc3(x)
        return x

# Establecer el dispositivo e inicializar el modelo
device = "cuda" if torch.cuda.is_available() else "cpu"
model_opt = CNN_Optimized(in_channels=3, num_classes=5).to(device)
print(model_opt)

CNN_Optimized(
  (conv1): Conv2d(3, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (bn1): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (pool1): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (conv2): Conv2d(16, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (bn2): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (pool2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (conv3): Conv2d(32, 48, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (bn3): BatchNorm2d(48, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (pool3): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (dropout): Dropout(p=0.2, inplace=False)
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (fc1): Linear(in_features=49152, out_features=70, bias=True)
  (fc2): Linear(in_features=70, out_features=35, bias=True)
  (fc3): Linear(in

In [ ]:
model_deep_opt = CNN_Optimized(in_channels=3, num_classes=5).to(device)

In [ ]:
model_deep_opt.load_state_dict(torch.load('model_deep_opt.pth', map_location=device))

<All keys matched successfully>

In [ ]:
model_deep_opt.eval()
print("Modelo cargado exitosamente.")

Modelo cargado exitosamente.


In [ ]:
print(model_deep_opt)

CNN_Optimized(
  (conv1): Conv2d(3, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (bn1): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (pool1): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (conv2): Conv2d(16, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (bn2): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (pool2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (conv3): Conv2d(32, 48, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (bn3): BatchNorm2d(48, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (pool3): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (dropout): Dropout(p=0.2, inplace=False)
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (fc1): Linear(in_features=49152, out_features=70, bias=True)
  (fc2): Linear(in_features=70, out_features=35, bias=True)
  (fc3): Linear(in

In [ ]:
# Realizar la predicción
with torch.no_grad():
    predicciones = model_deep_opt(img_futuro)
    _, etiquetas_predichas = torch.max(predicciones, dim=1)
    etiquetas_predichas = etiquetas_predichas.cpu().numpy()

In [ ]:
# Decodificar las etiquetas
etiquetas_decodificadas = labelencoder.inverse_transform(etiquetas_predichas)

In [ ]:
# Imprimir resultados
print("\nPredicciones:")
for nombre, etiqueta in zip(image_names, etiquetas_decodificadas):
    print(f"Imagen: {nombre} -> Clase predicha: {etiqueta}")


Predicciones:
Imagen: pinton1.jpg -> Clase predicha: pintones
Imagen: pinton4.jpg -> Clase predicha: pintones
Imagen: seco2.jpg -> Clase predicha: secos
Imagen: seco1.jpg -> Clase predicha: secos
Imagen: maduro3.jpg -> Clase predicha: maduros
Imagen: maduro6.jpg -> Clase predicha: maduros
Imagen: maduro4.jpg -> Clase predicha: maduros
Imagen: sobremaduro1.jpg -> Clase predicha: sobremaduros
Imagen: sobremaduro4.jpg -> Clase predicha: sobremaduros
Imagen: maduro1.jpg -> Clase predicha: maduros
Imagen: seco3.jpg -> Clase predicha: secos
Imagen: maduro5.jpg -> Clase predicha: maduros
Imagen: maduro2.jpg -> Clase predicha: maduros
Imagen: pinton6.jpg -> Clase predicha: pintones
Imagen: verde2.jpg -> Clase predicha: verdes
Imagen: verde1.jpg -> Clase predicha: verdes
Imagen: sobremaduro2.jpg -> Clase predicha: sobremaduros
Imagen: sobremaduro3.jpg -> Clase predicha: sobremaduros
Imagen: seco4.jpg -> Clase predicha: secos
Imagen: seco5.jpg -> Clase predicha: secos
